In [ ]:
!pip install pytextrank
from bs4 import BeautifulSoup
import string
import requests
import re
import pytextrank
import spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def remove_tags(html):
  soup = BeautifulSoup(html, "html.parser")
  for data in soup(['style', 'script']):
      data.decompose()
  return ' '.join(soup.stripped_strings)

In [ ]:
imsdb_url="https://imsdb.com/all-scripts.html"

In [ ]:
try:
  url=imsdb_url+str(imsdb_url)
  page_request=requests.get(url)
except:
  print("error")
soup = BeautifulSoup(page_request.content, 'html.parser')
all_td=soup.find_all('td')
all_hrefs=re.findall('.*href="/Movie Scripts.*html"',str(all_td[5]))
all_movies_a_tag=str(all_hrefs[5]).split("</a>")
all_htmls=[]
for movie in all_movies_a_tag:
  href_split=re.split('.*href="',movie)
  quote_split=href_split[1].split('"')
  all_htmls.append("https://imsdb.com"+str(quote_split[0]))

In [ ]:
print(all_htmls)

['https://imsdb.com/Movie Scripts/10 Things I Hate About You Script.html', 'https://imsdb.com/Movie Scripts/12 Script.html', 'https://imsdb.com/Movie Scripts/12 and Holding Script.html', 'https://imsdb.com/Movie Scripts/12 Monkeys Script.html', 'https://imsdb.com/Movie Scripts/12 Years a Slave Script.html', 'https://imsdb.com/Movie Scripts/127 Hours Script.html', 'https://imsdb.com/Movie Scripts/1492: Conquest of Paradise Script.html', 'https://imsdb.com/Movie Scripts/15 Minutes Script.html', 'https://imsdb.com/Movie Scripts/17 Again Script.html', 'https://imsdb.com/Movie Scripts/187 Script.html', 'https://imsdb.com/Movie Scripts/2001: A Space Odyssey Script.html', 'https://imsdb.com/Movie Scripts/2012 Script.html', 'https://imsdb.com/Movie Scripts/25th Hour Script.html', 'https://imsdb.com/Movie Scripts/28 Days Later Script.html', 'https://imsdb.com/Movie Scripts/30 Minutes or Less Script.html', 'https://imsdb.com/Movie Scripts/42 Script.html', 'https://imsdb.com/Movie Scripts/44 Inch

In [ ]:
all_movies=[]

In [ ]:
for movie in all_htmls:
  movie_dict={}
  try:
    request=requests.get(movie)
  except:
    print("error")
  soup=BeautifulSoup(request.content,'html.parser')
  all_td=soup.find_all("td")
  try:
    all_td_td=all_td[5].find_all("td")
    movie_dict["title"]=str(all_td_td[75]).split("<h1>")[1].split("</h1>")[0].replace(" Script","")
    movie_script_link=re.findall('.*href="/scripts.*html',str(all_td_td[77]))
    movie_script_link_split=movie_script_link[0].split('href="')
    movie_dict["script_link"]="https://imsdb.com"+movie_script_link_split[1]
    try:
      all_rating=re.findall(".*out of 10.*",str(all_td_td[74]))
      movie_dict["imdb_rating"]=str(all_rating[0]).split('gif"/> (')[1].split(" ")[0]
      movie_dict["user_rating"]=str(all_rating[1]).split('gif"/> (')[1].split(" ")[0]
    except:
      movie_dict["imdb_rating"]=0
      movie_dict["user_rating"]=0
      pass
    try:
      all_href=re.findall('.*href.*',str(all_td_td[74]))
      soup=BeautifulSoup(all_href[0],'html.parser')
      all_writers=[]
      for writer in soup.find_all("a"):
        all_writers.append(writer.text)
      movie_dict["crew"]=all_writers
      soup=BeautifulSoup(all_href[1],'html.parser')
      all_genres=[]
      for genres in soup.find_all("a"):
        all_genres.append(genres.text)
      movie_dict["genres"]=all_genres
    except:
      movie_dict["crew"]=[]
      movie_dict["genres"]=[]
      pass

    all_movies.append(movie_dict)
  except:
    continue

In [ ]:
nlp=spacy.load("en_core_web_sm")
nlp.add_pipe("textrank")  

In [ ]:
type(all_movies)

list

In [ ]:
len=0
for movie in all_movies:
  try:
    request=requests.get(movie["script_link"])
  except:
    print("error")
  soup=BeautifulSoup(request.content,'html.parser')
  all_pre=soup.find_all("pre")
  try:
    script=remove_tags(str(all_pre[0].find_all("pre")))
    movie["script"]=script
    movie["script_summary"]=str(nlp(script))
  except:
    pass
  len+=1

In [ ]:
import json

In [ ]:
output_file =  open('movie_scripts_imsdb.txt','w', encoding="UTF-8")
output_file.write(json.dumps({"movie_scripts_imsdb":all_movies}))

81064003

# New Section

In [ ]:
input_file =  open('movie_scripts_imsdb.txt')

In [ ]:
movie_scripts=json.load(input_file)

In [ ]:
id=0
error_id=[]
for movie in movie_scripts["movie_scripts_imsdb"]:
  title=movie['title']
  try:
    response=requests.get("http://omdbapi.com/?apikey=9a4b9de0&t="+title)
    response_dict=response.json()
    movie["cast"]=response_dict["Actors"].split(",")
    movie["crew"]=list(set(movie["crew"]+response_dict["Director"].split(",")))
    movie["runtime"]=response_dict["Runtime"].split(" ")[0]
    movie["imdbID"]=response_dict["imdbID"]
  except:
    error_id.append(id)
  id+=1
  print(id)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
for movie in movie_scripts["movie_scripts_imsdb"]:
  if movie_scripts["movie_scripts_imsdb"].index(movie) in error_id:
    try:
      response=requests.get("http://omdbapi.com/?apikey=3738af91&t="+title)
      response_dict=response.json()
      movie["cast"]=response_dict["Actors"].split(",")
      movie["crew"]=list(set(movie["crew"]+response_dict["Director"].split(",")))
      movie["runtime"]=response_dict["Runtime"].split(" ")[0]
      movie["imdbID"]=response_dict["imdbID"]
    except Exception as err:
      print(err)

In [ ]:
output_file =  open('movie_scripts_imsdb_omdb.txt','w', encoding="UTF-8")
output_file.write(json.dumps(movie_scripts))

81207094

# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
input_file =  open('movie_scripts_imsdb_omdb.txt')

In [ ]:
movie_scripts_omdb=json.load(input_file)

In [ ]:
error_id=[]
id=0
for movie in movie_scripts_omdb["movie_scripts_imsdb"]:
  url="https://api.themoviedb.org/3/find/"+movie['imdbID']+"?api_key=fb1488ce9d86e2c2a89c36bc44da34e6&language=en-US&external_source=imdb_id"
  print(url)
  try:
    response=requests.get(url)
    response_dict=response.json()
    movie["movie_id"]=response_dict["movie_results"][0]["id"]
    print(movie["movie_id"])
  except Exception as err:
    error_id.append(id)
  id+=1

https://api.themoviedb.org/3/find/tt0147800?api_key=fb1488ce9d86e2c2a89c36bc44da34e6&language=en-US&external_source=imdb_id
4951
https://api.themoviedb.org/3/find/tt0488478?api_key=fb1488ce9d86e2c2a89c36bc44da34e6&language=en-US&external_source=imdb_id
20714
https://api.themoviedb.org/3/find/tt0417385?api_key=fb1488ce9d86e2c2a89c36bc44da34e6&language=en-US&external_source=imdb_id
18164
https://api.themoviedb.org/3/find/tt0114746?api_key=fb1488ce9d86e2c2a89c36bc44da34e6&language=en-US&external_source=imdb_id
63
https://api.themoviedb.org/3/find/tt2024544?api_key=fb1488ce9d86e2c2a89c36bc44da34e6&language=en-US&external_source=imdb_id
76203
https://api.themoviedb.org/3/find/tt1542344?api_key=fb1488ce9d86e2c2a89c36bc44da34e6&language=en-US&external_source=imdb_id
44115
https://api.themoviedb.org/3/find/tt0103594?api_key=fb1488ce9d86e2c2a89c36bc44da34e6&language=en-US&external_source=imdb_id
1492
https://api.themoviedb.org/3/find/tt0179626?api_key=fb1488ce9d86e2c2a89c36bc44da34e6&language=e

In [ ]:
error_id=[]
id=0
for movie in movie_scripts_omdb["movie_scripts_imsdb"]:
  if movie.get("movie_id"):
    url="https://api.themoviedb.org/3/movie/"+str(movie['movie_id'])+"?api_key=fb1488ce9d86e2c2a89c36bc44da34e6&language=en-US"
    print(url)
    try:
      response=requests.get(url)
      response_dict=response.json()
      movie["revenue"]=response_dict["revenue"]
      movie["budget"]=response_dict["budget"]
      print(movie["revenue"],movie["budget"])
    except Exception as err:
      error_id.append(id)
  else:
    movie["revenue"]=0
    movie["budget"]=0
  id+=1

https://api.themoviedb.org/3/movie/4951?api_key=fb1488ce9d86e2c2a89c36bc44da34e6&language=en-US
53478166 16000000
https://api.themoviedb.org/3/movie/20714?api_key=fb1488ce9d86e2c2a89c36bc44da34e6&language=en-US
7537453 4000000
https://api.themoviedb.org/3/movie/18164?api_key=fb1488ce9d86e2c2a89c36bc44da34e6&language=en-US
314017 400000
https://api.themoviedb.org/3/movie/63?api_key=fb1488ce9d86e2c2a89c36bc44da34e6&language=en-US
168841459 29000000
https://api.themoviedb.org/3/movie/76203?api_key=fb1488ce9d86e2c2a89c36bc44da34e6&language=en-US
187000000 20000000
https://api.themoviedb.org/3/movie/44115?api_key=fb1488ce9d86e2c2a89c36bc44da34e6&language=en-US
35692920 18000000
https://api.themoviedb.org/3/movie/1492?api_key=fb1488ce9d86e2c2a89c36bc44da34e6&language=en-US
7191399 47000000
https://api.themoviedb.org/3/movie/2749?api_key=fb1488ce9d86e2c2a89c36bc44da34e6&language=en-US
56359980 60000000
https://api.themoviedb.org/3/movie/16996?api_key=fb1488ce9d86e2c2a89c36bc44da34e6&language=

In [ ]:
output_file =  open('movie_scripts_imsdb_omdb_moviedb.txt','w', encoding="UTF-8")
output_file.write(json.dumps(movie_scripts_omdb))

81272662

In [ ]:
print(movie_scripts_omdb["movie_scripts_imsdb"][0])

{'title': '10 Things I Hate About You', 'script_link': 'https://imsdb.com/scripts/10-Things-I-Hate-About-You.html', 'imdb_rating': '7', 'user_rating': '8.76', 'crew': ['Karen McCullah Lutz', 'William Shakespeare', 'Kirsten Smith', 'Gil Junger'], 'genres': ['Comedy', 'Romance'], 'script': '[ TEN THINGS I HATE ABOUT YOU written by Karen McCullah Lutz & Kirsten Smith based on \'Taming of the Shrew" by William Shakespeare Revision November 12, 1997 PADUA HIGH SCHOOL - DAY Welcome to Padua High School,, your typical urban-suburban \r\n          high school in Portland, Oregon.  Smarties, Skids, Preppies, \r\n          Granolas. Loners, Lovers, the In and the Out Crowd rub sleep \r\n          out of their eyes and head for the main building. PADUA HIGH PARKING LOT - DAY KAT STRATFORD, eighteen, pretty -- but trying hard not to be \r\n          -- in a baggy granny dress and glasses, balances a cup of \r\n          coffee and a backpack as she climbs out of her battered, \r\n          baby bl

In [ ]:
import pandas as pd
import json
import csv

In [ ]:
input_file =  open('movie_scripts_imsdb_omdb_moviedb.txt')
movie_script_json=json.load(input_file)

In [ ]:
myFile = open('movie_scripts_imsdb_omdb_moviedb.csv', 'w')
writer = csv.DictWriter(myFile, fieldnames=['title', 'script_link', 'imdb_rating', 'user_rating', 'crew', 'genres', 'script', 'script_summary', 'cast', 'runtime', 'imdbID', 'movie_id', 'revenue', 'budget'])
writer.writeheader()
writer.writerows(movie_script_json["movie_scripts_imsdb"])
myFile.close()